<a href="https://colab.research.google.com/github/rupakdas18/SuperGlue-tasks-using-BERT/blob/main/boolq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 4.3MB/s 
     |████████████████████████████████| 3.3MB 42.5MB/s 
     |████████████████████████████████| 901kB 50.3MB/s 


In [ ]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
learning_rate = 2e-5
optimizer = AdamW(model.parameters(), lr=learning_rate, correct_bias=False)

In [ ]:
def load_data(tokenizer, questions, passages, max_length):
    """Encode the question/passage pairs into features than can be fed to the model."""
    input_ids = []
    attention_masks = []

    for question, passage in zip(questions, passages):
        encoded_data = tokenizer.encode_plus(question, passage, max_length=max_length, pad_to_max_length=True, truncation_strategy="longest_first")
        encoded_pair = encoded_data["input_ids"]
        attention_mask = encoded_data["attention_mask"]

        input_ids.append(encoded_pair)
        attention_masks.append(attention_mask)
    return np.array(input_ids), np.array(attention_masks)

In [ ]:
train_data_df = pd.read_csv("data/train.csv", header=None, names=('question','passage','label','idx'))
dev_data_df = pd.read_csv("data/val.csv", header=None,names=('question','passage','label','idx'))


passages_train = train_data_df.passage.values
questions_train = train_data_df.question.values
answers_train = train_data_df.label.values.astype(int)
print(answers_train)

passages_dev = dev_data_df.passage.values
questions_dev = dev_data_df.question.values
answers_dev = dev_data_df.label.values.astype(int)

# Encoding data
max_seq_length = 256
input_ids_train, attention_masks_train = load_data(tokenizer, questions_train, passages_train, max_seq_length)
input_ids_dev, attention_masks_dev = load_data(tokenizer, questions_dev, passages_dev, max_seq_length)

train_features = (input_ids_train, attention_masks_train, answers_train)
dev_features = (input_ids_dev, attention_masks_dev, answers_dev)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[1 1 1 ... 1 0 0]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# Building Dataloaders
batch_size = 32

train_features_tensors = [torch.tensor(feature, dtype=torch.long) for feature in train_features]
dev_features_tensors = [torch.tensor(feature, dtype=torch.long) for feature in dev_features]

train_dataset = TensorDataset(*train_features_tensors)
dev_dataset = TensorDataset(*dev_features_tensors)

train_sampler = RandomSampler(train_dataset)
dev_sampler = SequentialSampler(dev_dataset)

train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
dev_dataloader = DataLoader(dev_dataset, sampler=dev_sampler, batch_size=batch_size)

In [ ]:
import time
from transformers import get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

# Number of EPOCHS
EPOCHS = 4
total_steps = len(train_dataloader) * EPOCHS
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
def accuracy(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [ ]:
#for _ in tqdm(range(epochs), desc="Epoch"):
for epoch in range(EPOCHS):

  # Training
  model.train()

  # Reset the total loss and accuracy for this epoch.
  total_train_loss = 0
  total_train_acc = 0
  
  # Measure how long the training epoch takes.
  start = time.time()

  for step, batch in enumerate(train_dataloader):
      # Unpack this training batch from our dataloader.
      input_ids = batch[0].to(device)
      attention_masks = batch[1].to(device)
      labels = batch[2].to(device)  

      #clear any previously calculated gradients before performing a backward pass
      optimizer.zero_grad()

      #result = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels)
      loss, prediction = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels).values()
      #loss = result.loss
      #logits = result.logits
      acc = accuracy(prediction, labels)

      # Accumulate the training loss and accuracy over all of the batches so that we can
      # calculate the average loss at the end
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

      # Perform a backward pass to calculate the gradients.
      loss.backward()

      # Clip the norm of the gradients to 1.0.
      # This is to help prevent the "exploding gradients" problem.
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      # Update parameters and take a step using the computed gradient.
      optimizer.step()

      # Update the learning rate.
      scheduler.step()

  # Calculate the average accuracy and loss over all of the batches.
  train_acc  = total_train_acc/len(train_dataloader)
  train_loss = total_train_loss/len(train_dataloader)

  # Put the model in evaluation mode
  model.eval()

  total_val_acc  = 0
  total_val_loss = 0

  with torch.no_grad():
    for batch in dev_dataloader:

      #clear any previously calculated gradients before performing a backward pass
      optimizer.zero_grad()

      # Unpack this validation batch from our dataloader.
      input_ids = batch[0].to(device)
      attention_masks = batch[1].to(device)
      labels = batch[2].to(device) 

      #Get the loss and prediction
      loss, prediction = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels).values()

      # Calculate the accuracy for this batch
      acc = accuracy(prediction, labels)

      # Accumulate the validation loss and Accuracy
      total_val_loss += loss.item()
      total_val_acc  += acc.item()

  # Calculate the average accuracy and loss over all of the batches.
  val_acc  = total_val_acc/len(dev_dataloader)
  val_loss = total_val_loss/len(dev_dataloader)

  end = time.time()
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)

  print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

Epoch 1: train_loss: 0.6424 train_acc: 0.6420 | val_loss: 0.6201 val_acc: 0.6678
00:02:20.83
Epoch 2: train_loss: 0.5224 train_acc: 0.7470 | val_loss: 0.6166 val_acc: 0.6914
00:02:21.13
Epoch 3: train_loss: 0.3635 train_acc: 0.8431 | val_loss: 0.7198 val_acc: 0.6880
00:02:21.15
Epoch 4: train_loss: 0.2523 train_acc: 0.9036 | val_loss: 0.8226 val_acc: 0.6984
00:02:21.20


In [ ]:
# Load Test data
test_data_df = pd.read_csv("data/train.csv", header=None, names=('passage','question','idx'))

In [ ]:
passage_list = test_data_df['passage'].to_list()
question_list = test_data_df['question'].to_list()
#pair_list = [(question_list[i], passage_list[i]) for i in range(0, len(question_list))]


In [ ]:
# Predict the answer
def predict(question, passage):
  sequence = tokenizer.encode_plus(question, passage, return_tensors="pt")['input_ids'].to(device)
  logits = model(sequence)[0]
  probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
  proba_yes = round(probabilities[1], 3)
  proba_no = round(probabilities[0], 3)
  print(f"Question: {question}, True: {proba_yes}, False: {proba_no}")

In [ ]:
for i in range(10):
  question = question_list[i]
  passage = passage_list[i]
  predict(question,passage)

Question: question, True: 0.43, False: 0.57
Question: is the first series 20 euro note still legal tender, True: 0.97, False: 0.03
Question: do the champions league winners get automatic qualification for next year's tournament, True: 0.98, False: 0.02
Question: can a bull snake kill a small dog, True: 0.97, False: 0.03
Question: are all nba playoff games best of 7, True: 0.03, False: 0.97
Question: can i use my train ticket on the tram in manchester, True: 0.99, False: 0.01
Question: is hunger games mockingjay part 2 the last movie, True: 0.1, False: 0.9
Question: can you run methanol in a gas engine, True: 0.99, False: 0.01
Question: do all captains go down with the ship, True: 0.33, False: 0.67
Question: was oceans 8 based on a true story, True: 0.62, False: 0.38
